# INFORMATION RETRIEVAL PROJECT
# 5. RESULTS OF ANALYSIS OF GENDER STEREOTYPES BY YEARS 

---
## Gender stereotypes in parliamentary speeches

In word embedding models, each word is assigned to a high-dimensional vector such that the geometry of the vectors captures semantic relations between the words – e.g. vectors being closer together has been shown to correspond to more similar words. Recent works in machine learning demonstrate that word embeddings also capture common stereotypes, as these stereotypes are likely to be present, even if subtly, in the large corpora of training texts. These stereotypes are automatically learned by the embedding algorithm and could be problematic in many context if the embedding is then used for sensitive applications such as search rankings, product recommendations, or translations. An important direction of research is on developing algorithms to debias the word embeddings.

This project aims to use the word embeddings to study historical trends – specifically trends in the gender and ethnic stereotypes in the Italian parliamentary speeches from 1948 to 2020.

In [60]:
import numpy as np
import pandas as pd
import gensim
from gensim.models import KeyedVectors
from gensim.models import Word2Vec
import os
from tqdm.auto import tqdm
from collections import defaultdict, OrderedDict

from INFORET_project import load_embed_model
from INFORET_project import YEARS, WORDS_GROUP # INFORMATION RETRIEVAL PROJECT
from INFORET_project.utils import to_list

---

## 1) TOP 6 MOST AND LESS BIASED GROUPS 
### (according to ECT, ANALOGIES BIAS and WEAT)

In [118]:
%store -r results

In [119]:
results

,EAB_1948_1968,EAB_avg_gender_1948_1968,ECT_1948_1968,WEAT_1948_1968,EAB_1968_1985,EAB_avg_gender_1968_1985,ECT_1968_1985,WEAT_1968_1985,EAB_1985_2000,EAB_avg_gender_1985_2000,ECT_1985_2000,WEAT_1985_2000,EAB_2000_2020,EAB_avg_gender_2000_2020,ECT_2000_2020,WEAT_2000_2020
0,gendered_words,gendered_words,adj_appearence,family,gendered_words,gendered_words,family,family,gendered_words,gendered_words,gendered_words,family,gendered_words,gendered_words,gendered_words,family
1,active,intelligence,gendered_words,gendered_words,active,family,active,gendered_words,family,family,kindness,gendered_words,passive,family,kindness,gendered_words
2,adj_appearence,adj_appearence,dumbness,dumbness,passive,adj_appearence,female_stereotypes,dumbness,passive,career,career,passive,family,rage,passive,passive
3,male_stereotypes,family,rage,passive,kindness,dumbness,intelligence,passive,adj_appearence,passive,female_stereotypes,active,rage,passive,career,dumbness
4,kindness,career,female_stereotypes,rage,intelligence,intelligence,passive,rage,rage,kindness,family,dumbness,intelligence,career,intelligence,rage
5,passive,male_stereotypes,family,adj_appearence,adj_appearence,female_stereotypes,adj_appearence,active,kindness,adj_appearence,rage,rage,adj_appearence,dumbness,family,kindness
6,family,passive,career,kindness,family,kindness,career,adj_appearence,active,female_stereotypes,intelligence,kindness,kindness,female_stereotypes,active,female_stereotypes
7,intelligence,female_stereotypes,passive,female_stereotypes,male_stereotypes,male_stereotypes,kindness,kindness,female_stereotypes,intelligence,adj_appearence,male_stereotypes,male_stereotypes,male_stereotypes,male_stereotypes,adj_appearence
8,female_stereotypes,kindness,kindness,career,female_stereotypes,passive,dumbness,male_stereotypes,career,active,dumbness,female_stereotypes,active,kindness,female_stereotypes,male_stereotypes
9,rage,rage,intelligence,male_stereotypes,rage,career,gendered_words,female_stereotypes,male_stereotypes,dumbness,passive,intelligence,female_stereotypes,intelligence,rage,career


In [130]:
# intersection without EAB

for year in YEARS:
    print(f'\n\nYEAR: {year}\n')
    
    print('TOP 6 MOST BIASED:')
    display( set.intersection(
        set( to_list(results[f'ECT_{year}'][:6]) ),
        set( to_list(results[f'WEAT_{year}'][:6]) )
    ))
    
    print('\nTOP 6 LEAST BIASED:')
    
    display( set.intersection(
        set( to_list(results[f'ECT_{year}'][-6:]) ),
        set( to_list(results[f'WEAT_{year}'][-6:]) )
    ))



YEAR: 1948_1968

TOP 6 MOST BIASED:


{'adj_appearence', 'dumbness', 'family', 'gendered_words', 'rage'}


TOP 6 LEAST BIASED:


{'active', 'career', 'intelligence', 'kindness', 'male_stereotypes'}



YEAR: 1968_1985

TOP 6 MOST BIASED:


...

{'adj_appearence', 'intelligence', 'male_stereotypes'}



YEAR: 2000_2020

TOP 6 MOST BIASED:


{'family', 'gendered_words', 'kindness', 'passive'}


TOP 6 LEAST BIASED:


{'active', 'adj_appearence', 'female_stereotypes', 'male_stereotypes'}

In [131]:
# intersection with EAB

for year in YEARS:
    print(f'\n\nYEAR: {year}\n')
    
    print('TOP 6 MOST BIASED:')
    display( set.intersection(
        set( to_list(results[f'EAB_{year}'][:6]) ),
        set( to_list(results[f'ECT_{year}'][:6]) ),
        set( to_list(results[f'WEAT_{year}'][:6]) )
    ))
    
    display( set.intersection(
        set( to_list(results[f'EAB_avg_gender_{year}'][:6]) ),
        set( to_list(results[f'ECT_{year}'][:6]) ),
        set( to_list(results[f'WEAT_{year}'][:6]) )
    ))
    
    print('\nTOP 6 LEAST BIASED:')
    
    display( set.intersection(
        set( to_list(results[f'EAB_{year}'][-6:]) ),
        set( to_list(results[f'ECT_{year}'][-6:]) ),
        set( to_list(results[f'WEAT_{year}'][-6:]) )
    ))
    
    display( set.intersection(
        set( to_list(results[f'EAB_avg_gender_{year}'][-6:]) ),
        set( to_list(results[f'ECT_{year}'][-6:]) ),
        set( to_list(results[f'WEAT_{year}'][-6:]) )
    ))



YEAR: 1948_1968

TOP 6 MOST BIASED:


{'adj_appearence', 'gendered_words'}

{'adj_appearence', 'family', 'gendered_words'}


TOP 6 LEAST BIASED:


{'career', 'intelligence'}

...

{'family', 'gendered_words', 'passive'}

{'family', 'gendered_words', 'passive'}


TOP 6 LEAST BIASED:


{'active', 'female_stereotypes', 'male_stereotypes'}

{'active', 'adj_appearence', 'female_stereotypes', 'male_stereotypes'}

## 2) TOP 3 BIASED WORDS PER GROUP

In [124]:
%store -r EAB_top_bias_NO_avg_gender
%store -r EAB_top_bias_avg_gender
%store -r ECT_top_bias

In [127]:
def show_top_biased_words_intersection(year, topn=3, verbose=True):
    
    if verbose:
        print(f'YEAR: {year}\n\nTOP {topn} BIASED WORDS\n\n')
    
    %store -r EAB_top_bias_NO_avg_gender
    %store -r EAB_top_bias_avg_gender
    %store -r ECT_top_bias
    
    top_biased_words_intersection_NO_avg = defaultdict(list)
    top_biased_words_intersection_avg = defaultdict(list)
    
    for group in WORDS_GROUP:
        if verbose:
            print(f'GROUP OF WORDS: {group}')
        
        top_biased_words_intersection_NO_avg[group] = set.intersection(
            set( to_list( [ i[0] for i in EAB_top_bias_avg_gender[year][group] ][:topn]) ), 
            set( to_list( [ i[0] for i in ECT_top_bias[year][group] ][:topn] )) 
            )
        
        top_biased_words_intersection_avg[group] = set.intersection(
            set( to_list( [ i[0] for i in EAB_top_bias_NO_avg_gender[year][group] ][:topn]) ),
            set( to_list( [ i[0] for i in ECT_top_bias[year][group] ][:topn] )) 
            )
        
        if verbose:
            display( top_biased_words_intersection_NO_avg[group] )
            display( top_biased_words_intersection_avg[group] )
        
    return top_biased_words_intersection_NO_avg, top_biased_words_intersection_avg

In [157]:
_,_ = show_top_biased_words_intersection(YEARS[3])

YEAR: 2000_2020

TOP 3 BIASED WORDS


GROUP OF WORDS: adj_appearence


{'splendido'}

{'splendido'}

GROUP OF WORDS: family


{'bambino', 'figlio'}

...

{'insicuro'}

set()

GROUP OF WORDS: male_stereotypes


{'crudele'}

{'intelligente'}

## 3) DISPLAY ANALOGIES FOR THE TOP 3 BIASED WORDS 

### 1. ANALOGIES BIAS ONLY

In [63]:
from INFORET_project.data import gendered_neutral_words
from INFORET_project import Analogies_Distance_Bias

In [111]:
def show_top_biased_analogies(year, topn=3, intersection=False):

    print(f'YEAR: {year}\n\n')
    model = load_embed_model(year)
    
    if intersection:
        words,_ = show_top_biased_words_intersection(year, verbose=False)
    
    for group in WORDS_GROUP:
        
        score = AEB(model.wv,
                    gender_female='donna',
                    gender_male='uomo',
                    type_most_similar = 'cosmul')
        
        try:
            if intersection:
                    top_bias = score.get_top_bias(positive_word=words[group], verbose=False)
            else:
                top_bias = score.get_top_bias(pred_positive_word=group, verbose=False)
            
            print(f'GROUP OF WORDS: {group}\nTOP 3 BIASED WORDS:')
            score.print_top_analogies(topn=topn)
        except:
            pass

In [112]:
show_top_biased_analogies(YEARS[0])

YEAR: 1948_1968


GROUP OF WORDS: adj_appearence
TOP 3 BIASED WORDS:

Word: brutto

Positive gender: male


[('brutto', 0.5006075524125156),
 ('uomo', 0.5005223682383805),
 ('bruttare', 0.5004065489840138),
 ('hobby', 0.5003803105203399),
 ('araldo', 0.5003687344089883),
 ('inquisitore', 0.500366682238291),
 ('eclettico', 0.5003541856268412),
 ('illustre', 0.5003536100728226),
 ('lapsus', 0.500351871846717),
 ('operetta', 0.5003493893461514)]

Positive gender: female


[('brutto', 0.5005600859855832),
 ('donna', 0.5004986385057769),
 ('ragazza', 0.5004032723367208),
 ('casalingo', 0.500328544216722),
 ('bello', 0.5003203944201551),
 ('coniugato', 0.5003145616937776),
 ('bambina', 0.5003131384860301),
 ('madre', 0.5003053465979763),
 ('traviata', 0.5002912342907856),
 ('passeggiatrice', 0.5002908317425176)]


Word: rozzo

Positive gender: male


...

[('saggio', 0.5005269346995213),
 ('donna', 0.5004121814954103),
 ('disoccupato', 0.5002716132600364),
 ('lavoratrice', 0.5002685311200208),
 ('ragazza', 0.500265076097943),
 ('madre', 0.5002649803855453),
 ('avventurarci', 0.5002614996636924),
 ('giusto', 0.500246854213147),
 ('coniugato', 0.5002436268110554),
 ('lcro', 0.5002428231461102)]


Word: intelligente

Positive gender: male


[('intelligente', 0.5006844411192348),
 ('uomo', 0.5005816357571263),
 ('sagace', 0.5004606794904273),
 ('preveggente', 0.5004554461621578),
 ('inquisitore', 0.500452794053605),
 ('integerrimo', 0.5004461496612003),
 ('ammiratore', 0.5004443921501132),
 ('arguto', 0.5004417630574339),
 ('abile', 0.5004399159922122),
 ('amicare', 0.5004333573659004)]

Positive gender: female


[('intelligente', 0.5004832164714882),
 ('donna', 0.5004810338957744),
 ('casalingo', 0.5003261691987989),
 ('lavoratrice', 0.5003208064143282),
 ('ragazza', 0.5003165306304982),
 ('risparmiatrici', 0.5003073572147586),
 ('traviata', 0.5003003560660408),
 ('parsimonioso', 0.5002968184527503),
 ('madre', 0.5002935376583318),
 ('volenteroso', 0.5002883016756142)]

### 2. INTERSECTION BETWEEN ECT AND ANALOGIES BIAS

In [113]:
show_top_biased_analogies(YEARS[0], intersection=True)

YEAR: 1948_1968


GROUP OF WORDS: adj_appearence
TOP 3 BIASED WORDS:

Word: carino

Positive gender: male


[('carino', 0.5005172475390353),
 ('uomo', 0.5004332181520663),
 ('abilitA', 0.5003347327363333),
 ('Iiberth', 0.5003334800018048),
 ('araldo', 0.5003281758217286),
 ('Eiar', 0.5003205540286989),
 ('chiazzare', 0.5003178634411751),
 ('opt', 0.5003178626960983),
 ('potentissimo', 0.5003108863626589),
 ('bonini', 0.5003107381042139)]

Positive gender: female


[('carino', 0.500650398403555),
 ('donna', 0.5004997780664112),
 ('nutrice', 0.5003819799889268),
 ('lir', 0.5003759527275314),
 ('manganare', 0.5003670921004102),
 ('erie', 0.500366181472612),
 ('ragazza', 0.5003655861061836),
 ('lavoratrice', 0.5003513395646892),
 ('autocolonne', 0.5003489961904586),
 ('madre', 0.5003463264801459)]

GROUP OF WORDS: family
TOP 3 BIASED WORDS:

Word: accudire

Positive gender: male


...

[('saggio', 0.5005269346995213),
 ('donna', 0.5004121814954103),
 ('disoccupato', 0.5002716132600364),
 ('lavoratrice', 0.5002685311200208),
 ('ragazza', 0.500265076097943),
 ('madre', 0.5002649803855453),
 ('avventurarci', 0.5002614996636924),
 ('giusto', 0.500246854213147),
 ('coniugato', 0.5002436268110554),
 ('lcro', 0.5002428231461102)]


Word: intelligente

Positive gender: male


[('intelligente', 0.5006844411192348),
 ('uomo', 0.5005816357571263),
 ('sagace', 0.5004606794904273),
 ('preveggente', 0.5004554461621578),
 ('inquisitore', 0.500452794053605),
 ('integerrimo', 0.5004461496612003),
 ('ammiratore', 0.5004443921501132),
 ('arguto', 0.5004417630574339),
 ('abile', 0.5004399159922122),
 ('amicare', 0.5004333573659004)]

Positive gender: female


[('intelligente', 0.5004832164714882),
 ('donna', 0.5004810338957744),
 ('casalingo', 0.5003261691987989),
 ('lavoratrice', 0.5003208064143282),
 ('ragazza', 0.5003165306304982),
 ('risparmiatrici', 0.5003073572147586),
 ('traviata', 0.5003003560660408),
 ('parsimonioso', 0.5002968184527503),
 ('madre', 0.5002935376583318),
 ('volenteroso', 0.5002883016756142)]